In [ ]:

'''
For the fine-tuning of the VGG-Face network for the emotion
recognition task, we investigated various options in our preliminary
analysis. We found that combining weight decay and dropout for regularization
gives the best results on the FER validation set. We carry
out a multi-stage fine-tuning. In the first stage, we fine-tune on the
FER public test set, and run weight updates for five epochs. In the second
stage, we update the upper layers (higher than layer 27) using'''

''' We then fine-tune the VGG-face model on FER 2013
dataset, using both the training and the public test set; during
training we use data augmentation by jittering the scale, flipping
and rotating the faces. The aim is to make the network more robust
to small misalignment of the faces. We also apply a strong dropout
on the last layer of the VGG (keeping only 5% of the nodes) to
prevent over-fitting. We achieve a performance of 71.2% on the
FER private test set, which is slightly higher than the previously
published results '''

'''During the training of the deep networks, we oversample the training
images by rotating them around their center by a random angle between 
−15° and 15°, and by circularly shifting the images in the horizontal and 
vertical directions by an amount no more than 20% of the image size. 
This approach helps our network to be more robust against alignment errors. 
In Fig. 2, we show the training curves of two stages of fine-tuning of the 
network with the FER dataset, where we set the learning rate and weight 
decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25].'''


### VGG16-Face model

In [ ]:
import VGG_FACE
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms

In [ ]:
model = VGG_FACE.VGG_FACE

model.load_state_dict(torch.load('VGG_FACE.pth'))

#how to test whether this is pretrained?
model.eval()


### Dataset: FERplus

In [ ]:


from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

imgTransform = transforms.Compose([transforms.Scale(256), #scale to 256x256
                                   transforms.CenterCrop(224), #crops the image at center to 224x224
                                   transforms.ToTensor()])
                                   #, #turn the jpg/pil/wahtever image into a tensor
                                   #transforms.Normalize(mean = [0.485, 0.456, 0.406], #normalize with these vals
                                                        #std=[0.229, 0.224, 0.225])])
                                    ##HOW TO GET NORMALIZED VALUES?
                                    #to add: jitter/rotate data augmentation, flipping, 
                                   
#this doesn't work because the data is organized w a csv file w prob distrib of labels 
#instead of a single ground truth
#see this paper: https://arxiv.org/pdf/1608.01041.pdf
trainset = ImageFolder(root= './all-data/FERPlus/data/FER2013Train/', transform = imgTransform) 
valset = ImageFolder(root='./all-data/FERPlus/data/FER2013Valid/', transform = imgTransform)

#i believe dataloader inherits from dataset
trainLoader = torch.utils.data.DataLoader(trainset, batch_size = 64, 
                                          shuffle = True, num_workers = 0)
valLoader = torch.utils.data.DataLoader(valset, batch_size = 64, 
                                       shuffle = False, num_workers = 0)

### set learning rate, loss, optimizer, all variable stuff

In [ ]:
#"where we set the learning rate and weight decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25]"
learningRate = 5e-4



# Definition of our network.
network = models.resnet34(pretrained = True) #resnet34 bc resnet50 takes too damn long
network.fc = nn.Linear(512, 2)
#basically this is just resnet34 with one fully connected linear layer that outputs two categories
#(resnet34 uses basic as its block so expansion = 1 and 1*512 = 512)

#Definition of our loss.
criterion = nn.CrossEntropyLoss()

# Definition of optimization strategy.
optimizer = optim.SGD(network.parameters(), lr = learningRate)

train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 3, use_gpu = True)